In [ ]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.9 MB/s eta 0:00:00


In [ ]:
import sys
from Bio import Entrez
import pandas as pd
import os
import scanpy as sc
import numpy as np

In [ ]:
cadata = sc.read("datasets/processed_cellcounrs.h5ad")

# **Create input files for R (Estimate)**

In [ ]:
!mkdir R_time

# Transpose the expression matrix
expression_matrix = cadata.X.T

# Get the gene and cell names
gene_names = cadata.var_names
cell_names = cadata.obs_names
lines = []

# Add the header line with cell names
header = "gene\t" + "\t".join(cell_names)
lines.append(header)

# Add the data lines with gene names and expression values
for i in range(expression_matrix.shape[0]):
    gene_name = gene_names[i]
    expression_values = "\t".join(expression_matrix[i].astype(str))
    line = gene_name + "\t" + expression_values
    lines.append(line)

# Write the lines to a text file in the Seurat_files directory
output_directory = "R_time"
os.makedirs(output_directory, exist_ok=True)
output_file = os.path.join(output_directory, "input_Estimate_format.txt")
with open(output_file, "w") as file:
    file.write("\n".join(lines))

# **Create input files for R (Genefu)**

In [ ]:
tumor_indices = np.where(cadata.obs['class'] == 'Tumor')[0]
expr_data = cadata.X[tumor_indices, :]

In [ ]:
# Save gene expression data to a CSV file
np.savetxt('R_time/gene_expression.csv', expr_data, delimiter=',')

# Save features.csv
features = pd.DataFrame({'GeneID': cadata.var_names})
features.to_csv('R_time/gene_names.csv', sep=',', index=False)

# Convert cell_names to a one-dimensional string array
cell_names_arr = np.array(cadata.obs_names)[tumor_indices].astype(str)

barcodes = pd.DataFrame({'CellID': cell_names_arr})
barcodes.to_csv('R_time/cell_names.csv', sep=',', index=False)

# **API for Entrez ID**

In [ ]:
Entrez.email = "s.alvanakis@gmail.com"

In [ ]:
dfs = pd.read_csv('R_time/gene_names.csv')

In [ ]:
genes_list = dfs["GeneID"].values.tolist()

In [ ]:
def get_entrez_gene_id(gene_name):
    handle = Entrez.esearch(db="gene", term=gene_name)
    record = Entrez.read(handle)
    handle.close()
    if len(record["IdList"]) > 0:
        return record["IdList"][0]
    else:
        return None


In [ ]:
entrez_gene_ids = []

In [ ]:
for gene_name in genes_list:
    entrez_gene_id = get_entrez_gene_id(gene_name)
    entrez_gene_ids.append(entrez_gene_id)


In [ ]:
dfs['Entrez_ID'] = entrez_gene_ids

In [ ]:
# Save DataFrame as a CSV file
dfs.to_csv('Entrez_id.csv', sep=',', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>